In [1]:
from datasets import load_dataset

ds = load_dataset("PatronusAI/financebench")

README.md: 0.00B [00:00, ?B/s]

financebench_merged.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [3]:
len(ds["train"])

150

In [6]:
for key in ds["train"][0].keys():
    print(key)

financebench_id
company
doc_name
question_type
question_reasoning
domain_question_num
question
answer
justification
dataset_subset_label
evidence
gics_sector
doc_type
doc_period
doc_link


In [7]:
ds["train"][0]["doc_link"]

'https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf'

In [13]:
ds["train"][0]["doc_name"]

'3M_2018_10K'

In [17]:
import requests

def download_pdf(url, folder_path, filename):
    response = requests.get(url)
    with open(f"{folder_path}/{filename}", "wb") as f:
        f.write(response.content)

folder_path = "financebench-pdfs"

failed_downloads = []

for training_sample in ds["train"]:
    doc_name, pdf_url = training_sample["doc_name"], training_sample["doc_link"]
    try:
        download_pdf(pdf_url, folder_path, f"{doc_name}.pdf")
    except:
        failed_downloads.append(f"{doc_name}")

if failed_downloads:
    print("Failed to download the following PDFs:")
    for failure in failed_downloads:
        print(failure)

Failed to download the following PDFs:
JOHNSON_JOHNSON_2022_10K
JOHNSON_JOHNSON_2022_10K
JOHNSON_JOHNSON_2022_10K
JOHNSON_JOHNSON_2022Q4_EARNINGS
JOHNSON_JOHNSON_2022Q4_EARNINGS
JOHNSON_JOHNSON_2023_8K_dated-2023-08-30
JOHNSON_JOHNSON_2023_8K_dated-2023-08-30
JOHNSON_JOHNSON_2023_8K_dated-2023-08-30
JOHNSON_JOHNSON_2023Q2_EARNINGS


In [19]:
import fitz
from pathlib import Path

DPI = 300
FORMAT = "png"

doc = fitz.open(f"financebench-pdfs/3M_2018_10K.pdf")

zoom = DPI / 72.0
mat = fitz.Matrix(zoom, zoom)

import time
start = time.time()
for i, page in enumerate(doc, start=1):
    pix = page.get_pixmap(matrix=mat, alpha=False)
    pix.save(f"financebench-pages/3M_2018_10K_{i}.png")

doc.close()
print(f"Saved {i} pages in {time.time() - start} seconds.")

Saved 160 pages in 25.219802856445312 seconds.


In [8]:
ds["train"][0]["question"]

'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.'

In [ ]:
ds["train"][0]["evidence"]

[{'evidence_text': 'Table of Contents \n3M Company and Subsidiaries\nConsolidated Statement of Cash Flow s\nYears ended December 31\n \n(Millions)\n \n2018\n \n2017\n \n2016\n \nCash Flows from Operating Activities\n \n \n \n \n \n \n \nNet income including noncontrolling interest\n \n$\n5,363 \n$\n4,869 \n$\n5,058 \nAdjustments to reconcile net income including noncontrolling interest to net cash\nprovided by operating activities\n \n \n \n \n \n \n \nDepreciation and amortization\n \n \n1,488 \n \n1,544 \n \n1,474 \nCompany pension and postretirement contributions\n \n \n(370) \n \n(967) \n \n(383) \nCompany pension and postretirement expense\n \n \n410 \n \n334 \n \n250 \nStock-based compensation expense\n \n \n302 \n \n324 \n \n298 \nGain on sale of businesses\n \n \n(545) \n \n(586) \n \n(111) \nDeferred income taxes\n \n \n(57) \n \n107 \n \n 7 \nChanges in assets and liabilities\n \n \n \n \n \n \n \nAccounts receivable\n \n \n(305) \n \n(245) \n \n(313) \nInventories\n \n \n(50